In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from io import BytesIO
from PIL import Image
import base64

# Load environment variables
load_dotenv(override=True)

# Ollama local settings
OLLAMA_BASE_URL = "http://localhost:11434/v1"
openai = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

# System prompt to give AI context
SYSTEM_PROMPT = """
You are a Technical Question Answering AI. You answer Python programming questions clearly and concisely.
Provide explanations, examples, and reasoning. Keep answers beginner-friendly when possible.
"""

# Optional: Tool to show Python code examples
def run_code_example(code: str):
    return f"Code example:\n{code}"

# Tool descriptor for Ollama
tools = [
    {
        "type": "function",
        "function": {
            "name": "run_code_example",
            "description": "Return Python code examples for explanations",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {"type": "string", "description": "Python code to illustrate the explanation"}
                },
                "required": ["code"]
            }
        }
    }
]

In [ ]:
def chat(message, history):
    # Format the history for Ollama
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]

    # Streaming response from Ollama
    stream = openai.chat.completions.create(
        model="llama3.2",
        messages=messages,
        stream=True,
        tools=tools
    )

    response_text = ""
    for chunk in stream:
        response_text += chunk.choices[0].delta.content or ""
        yield response_text

In [ ]:
# Function to add user message to the chatbot history
def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

In [ ]:

with gr.Blocks(css="""
    body { background-color: #FFFFFF; font-family: 'Arial', sans-serif; }
    .gradio-container { background-color: #FFFFFF; }

    .gradio-chatbot {
        background-color: #C0392B;  /* warm red */
        color: #FFFFFF; 
        font-weight: bold;
        border-radius: 15px;
        padding: 12px;
    }

    .gradio-chatbot .message.user {
        background-color: #E74C3C;  /* slightly lighter red */
        color: #FFFFFF;
        font-weight: bold;
        border-radius: 12px;
        padding: 10px;
    }

    .gradio-chatbot .message.assistant {
        background-color: #A93226;  /* slightly darker red */
        color: #FFFFFF;
        font-weight: bold;
        border-radius: 12px;
        padding: 10px;
    }

    .gradio-textbox textarea {
        background-color: #F1948A;  /* soft pink/red for input box */
        color: #000000; 
        font-weight: bold;
        border-radius: 12px;
        padding: 8px;
    }

    .gradio-button {
        background-color: #E74C3C; 
        color: #FFFFFF; 
        font-weight: bold;
        border-radius: 10px;
        padding: 8px 16px;
    }
""") as ui:

    chatbot = gr.Chatbot(height=500, type="messages")
    message = gr.Textbox(label="Ask your Python technical question:")

    message.submit(
        put_message_in_chatbot,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=[message, chatbot],
        outputs=chatbot
    )

ui.launch(inbrowser=True)
